<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/HuggingFace_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpa

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1157k      0  0:00:01  0:00:01 --:--:-- 1764k
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

In [4]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.8 MB 6.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 53.4 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 636 kB 56.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 7.6 MB/s 


In [5]:
# Pretokenizer KoNLPy
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

In [59]:
from transformers import *
import tensorflow as tf
from tqdm import tqdm
from typing import Optional, List, Union
from collections import Counter
from unicodedata import normalize
import urllib.request
import numpy as np
import pandas as pd
import json
import re
import os

In [7]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/"

Mounted at /content/gdrive/


In [8]:
songys = pd.read_csv("gdrive/My Drive/Colab Notebooks/HelloNewWorld/songysData.csv")

In [9]:
# 결측 row 제거하는 방법, 5232개
songys.dropna(how='any')
songys[:10]

Q                   A  label
0                   12시 땡!          하루가 또 가네요.      0
1              1지망 학교 떨어졌어           위로해 드립니다.      0
2             3박4일 놀러가고 싶다         여행은 언제나 좋죠.      0
3          3박4일 정도 놀러가고 싶다         여행은 언제나 좋죠.      0
4                  PPL 심하네          눈살이 찌푸려지죠.      0
5                SD카드 망가졌어  다시 새로 사는 게 마음 편해요.      0
6                  SD카드 안돼  다시 새로 사는 게 마음 편해요.      0
7           SNS 맞팔 왜 안하지ㅠㅠ    잘 모르고 있을 수도 있어요.      0
8  SNS 시간낭비인 거 아는데 매일 하는 중       시간을 정하고 해보세요.      0
9        SNS 시간낭비인데 자꾸 보게됨       시간을 정하고 해보세요.      0

In [10]:
texts = []
pairs = []

for sentence in songys['Q']:
    texts.append(sentence)

for sentence in songys['A']:
    pairs.append(sentence)

list(zip(texts, pairs))[:5]

[('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.')]

In [75]:
class KoNLPyPreTokenizer:
  def __init__(self, konlpy_tagger):
    self.konlpy_tagger = konlpy_tagger
  def __call__(self, sentence):
    return self.pre_tokenize(sentence)
  def pre_tokenize(self, sentence):
    return ' '.join(self.konlpy_tagger.morphs(sentence))

# sent_ko = '신종 코로나바이러스 감염증(코로나19) 사태가 심각합니다'
sent_ko = '안녕하세요? 만나서 반가워요'
pretok = KoNLPyPreTokenizer(Mecab())
print(f'input : {sent_ko}\npretok : {pretok(sent_ko)}')

input : 안녕하세요? 만나서 반가워요
pretok : 안녕 하 세요 ? 만나 서 반가워요


In [12]:
def prepare_pretokenized_corpus(pretokenized_path):
  with open(pretokenized_path, 'w', encoding='utf-8') as f:
    for line in texts:
      f.write(f'{pretok(line)}\n')
    for line in pairs:
      f.write(f'{pretok(line)}\n')

In [13]:
prepare_pretokenized_corpus(
    pretokenized_path = path+"ChatbotData.mecab.txt"
)

In [14]:
from tokenizers import BertWordPieceTokenizer

bert_wordpiece_tokenizer = BertWordPieceTokenizer()

bert_wordpiece_tokenizer.train(
    files = [path+'ChatbotData.mecab.txt'],
    vocab_size = 10000,
    min_frequency = 1,
    limit_alphabet = 1000,
    initial_alphabet = [],
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress = True,
    wordpieces_prefix = "##",
)
vocab = bert_wordpiece_tokenizer.get_vocab()
print(sorted(vocab, key=lambda x: vocab[x]))

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '_', 'a', 'b', 'c', 'd', 'g', 'j', 'k', 'l', 'n', 'o', 'p', 's', 'x', '~', 'ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ', 'ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ', 'ᅵ', 'ᆨ', 'ᆩ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆶ', 'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ', 'ᆻ', 'ᆼ', 'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ', '…', 'ㅊ', 'ㅋ', 'ㅎ', 'ㅜ', 'ㅠ', '##ᅥ', '##ᄄ', '##ᅢ', '##ᄉ', '##ᅩ', '##ᆨ', '##ᅡ', '##ᆼ', '##ᄒ', '##ᆻ', '##ᄆ', '##ᅮ', '##ᄂ', '##ᄊ', '##ᅳ', '##ᆯ', '##ᄃ', '##ᄐ', '##ᄅ', '##ᅧ', '##ᄋ', '##ᅭ', '##ᄁ', '##ᆷ', '##ᄌ', '##ᄀ', '##ᅦ', '##ᄇ', '##ᄏ', '##ᆺ', '##ᅵ', '##ᆫ', '##ᅨ', '##ᅣ', '##ᆸ', '##ᅬ', '##ᅪ', '##ᄑ', '##ᅯ', '##ᆽ', '##ᄎ', '##ᆲ', '##ᅲ', '##ᄈ', '##ᆩ', '##ᇀ', '##ᆾ', '##ᅴ', '##ᅱ', '##ᅫ', '##ᇂ', '##ᆭ', '##ᇁ', '##ᆰ', '##ᆮ', '##ᆹ', '##ᆪ', '##ㅠ', '##ᆱ', '##0', '##ᅤ', '##ᆶ', '##ㅎ

In [15]:
bert_wordpiece_tokenizer.save_model(path+'tokenizers/MecabBertWordPieceTokenizer/', 'chtbot')

['gdrive/My Drive/Colab Notebooks/HelloNewWorld/tokenizers/MecabBertWordPieceTokenizer/chtbot-vocab.txt']

In [57]:
def compose(tokens):
    return [normalize('NFKC', token) for token in tokens]

In [45]:
def initialize_alphabet(files, limit_alphabet, initial_alphabet, special_tokens, show_progress):
    counter = Counter()
    n_files = len(files)
    for i_file, file in enumerate(files):
        with open(file, encoding='utf-8') as f:
            lines = [line.strip() for line in f]
        if show_progress:
            iterator = tqdm(lines, desc=f'Initialize alphabet {i_file + 1}/{n_files}', total=len(lines))
        else:
            iterator = lines
        counter.update(Counter(char for line in iterator for char in line if char))
    del counter[' ']
    frequent_alphabets = sorted(counter, key=lambda x: -counter[x])
    frequent_alphabets = [alphabet for alphabet in frequent_alphabets if alphabet not in initial_alphabet]
    alphabets = special_tokens + frequent_alphabets
    alphabets = alphabets[:limit_alphabet]
    return alphabets

In [49]:
def train_vocab(files, vocab_size, min_frequency, show_progress, alphabets, tokenize):
    counter = Counter()
    n_files = len(files)
    for i_file, file in enumerate(files):
        with open(file, encoding='utf-8') as f:
            lines = [line.strip() for line in f]
        if show_progress:
            iterator = tqdm(lines, desc=f'Train vocab {i_file + 1}/{n_files}', total=len(lines))
        else:
            iterator = lines
        counter.update(Counter(sub for line in iterator for sub in tokenize(line)))
    frequent_vocab = sorted(counter, key=lambda x: -counter[x])
    frequent_vocab = [vocab for vocab in frequent_vocab
                      if (vocab not in alphabets) and (counter[vocab] >= min_frequency)]
    vocab = alphabets + frequent_vocab
    vocab = vocab[:vocab_size]
    return vocab

In [39]:
class KoNLPyWordPieceTokenizer:
    def __init__(self, konlpy_tagger, wordpieces_prefix="##", use_tag=False):
        self.konlpy_tagger = konlpy_tagger
        self.use_tag = use_tag
        if use_tag:
            def pretokenize(eojeol):
                return self.konlpy_tagger.pos(eojeol, join=True)
        else:
            def pretokenize(eojeol):
                return self.konlpy_tagger.morphs(eojeol)
        self.pretokenize = pretokenize
        self.prefix = wordpieces_prefix

    def tokenize(self, sent):
        def _tokenize(eojeol):
            split_tokens = self.pretokenize(eojeol)
            if len(split_tokens) <= 1:
                return split_tokens
            return [split_tokens[0]] + [f'{self.prefix}{sub}' for sub in split_tokens[1:]]

        split_tokens = []
        for eojeol in sent.split():
            split_tokens += _tokenize(eojeol)
        return split_tokens

    def token_to_alphabets(self, token):
        if token[:2] == self.prefix:
            token = token[2:]
        if self.use_tag:
            return list(token.rsplit('/')[0])
        return list(token)

In [66]:
class KoNLPyBertWordPieceTrainer:
    def __init__(self, konlpy_tagger, wordpieces_prefix="##", use_tag = False):
        konlpy_wordpiece = KoNLPyWordPieceTokenizer(
            konlpy_tagger,
            wordpieces_prefix,
            use_tag
        )
        self.tokenizer = konlpy_wordpiece

    def tokenize(self, sent):
        split_tokens = []
        for eojeol in sent.split():
            split_tokens += self.tokenizer.tokenize(eojeol)
        return split_tokens

    def train(
        self,
        files: Union[str, List[str]],
        vocab_size: int = 30000,
        min_frequency: int = 1,
        limit_alphabet: int = 1000,
        initial_alphabet: List[str] = [],
        special_tokens: List[str] = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
        show_progress: bool = True,
    ):
        if isinstance(files, str):
            files = [files]
        alphabets = initialize_alphabet(files, limit_alphabet, initial_alphabet, special_tokens, show_progress)
        self.vocab = train_vocab(files, vocab_size, min_frequency, show_progress, alphabets, self.tokenizer.tokenize)

    def save_model(self, directory: str, name: Optional[str] = None):
        if name is None:
            name = 'vocab.txt'
        else:
            name = f'{name}-vocab.txt'
        directory = os.path.abspath(directory)
        if not os.path.exists(directory):
            os.makedirs(directory)
        vocab_file = f'{directory}/{name}'
        with open(vocab_file, 'w', encoding='utf-8') as f:
            for subword in self.vocab:
                f.write(f'{subword}\n')
        print(f'[{vocab_file}]')

In [67]:
class KoNLPyBertTokenizer(BertTokenizer):
    def __init__(
        self,
        konlpy_wordpiece,
        vocab_file,
        do_lower_case=True,
        do_basic_tokenize=True,
        never_split=None,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        tokenize_chinese_chars=True,
        strip_accents=None,
        **kwargs
    ):
        super().__init__(
            vocab_file=vocab_file,
            do_lower_case=do_lower_case,
            do_basic_tokenize=do_basic_tokenize,
            never_split=never_split,
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            tokenize_chinese_chars=tokenize_chinese_chars,
            strip_accents=strip_accents
        )
        self.konlpy_wordpiece = konlpy_wordpiece

    def _tokenize(self, text):
        base_tokens = self.konlpy_wordpiece.tokenize(text)
        split_tokens = []
        for token in base_tokens:
            if token in self.vocab:
                split_tokens.append(token)
            else:
                split_tokens += self.konlpy_wordpiece.token_to_alphabets(token)
        return split_tokens

In [73]:
mecab_wordpiece_usetag_trainer = KoNLPyBertWordPieceTrainer(Mecab(), use_tag=True)
mecab_wordpiece_usetag_trainer.train(
    files = [path+'ChatbotData.txt']
)
mecab_wordpiece_usetag_trainer.save_model(path+'/tokenizers/BertStyleMecab/','usetag')

Train vocab 1/1: 100%|██████████| 11824/11824 [00:01<00:00, 7928.39it/s]


[/content/gdrive/My Drive/Colab Notebooks/HelloNewWorld/tokenizers/BertStyleMecab/usetag-vocab.txt]


In [76]:
konlpy_bert_usetag = KoNLPyBertTokenizer(
    konlpy_wordpiece = KoNLPyWordPieceTokenizer(Mecab(), use_tag=True),
    vocab_file = path+'tokenizers/BertStyleMecab/usetag-vocab.txt'
)

indices = konlpy_bert_usetag.encode(sent_ko)
tokens = [konlpy_bert_usetag.ids_to_tokens[ids] for ids in indices]
print(' '.join(compose(tokens)))

[CLS] 안녕/NNG ##하/XSV ##세요/EP+EF ##?/SF 만나/VV ##서/EC 반가워요/VA+EF [SEP]
